In [3]:
import pandas as pd

In [4]:
from zipfile import ZipFile
import pandas as pd
with ZipFile(f'D:/thermo_reader/TUM_first_pool_1_01_01_DDA-1h-R2-tryptic.zip', 'r') as zip_file:
  msms = pd.read_csv(zip_file.open('msms.txt'), sep='\t')
# Current PROSIT pipeline does not accomodate modified peptides, so we remove all of the oxidized peptides
msms = msms[msms['Modifications'] == 'Unmodified']


In [12]:
from fisher_py.data.business import Scan
import numpy as np
from fisher_py import RawFile

In [22]:
raw = RawFile("c:/20240708_chm134_Nadig_MAbPac-04A_Proteo06_Group01_Time00_biorep01_techrep01.raw")

In [32]:
for i in range(900,905):
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
    print(raw_scan.preferred_masses)
    print(raw_scan.preferred_intensities)

[499.9950866699219, 500.0125427246094, 500.05810546875, 500.0928649902344, 500.1105651855469, 500.15679931640625, 500.1703186035156, 500.18231201171875, 500.19830322265625, 500.281005859375, 500.2987365722656, 500.31414794921875, 500.32421875, 500.33892822265625, 500.3636474609375, 500.3742980957031, 500.3938903808594, 500.41595458984375, 500.4779968261719, 500.49725341796875, 500.5155944824219, 500.5794372558594, 500.60333251953125, 500.6146240234375, 500.69757080078125, 500.70660400390625, 500.7306213378906, 500.78070068359375, 500.793701171875, 500.8279113769531, 500.8531799316406, 500.8765563964844, 500.8954162597656, 500.9478759765625, 500.9952697753906, 501.0429992675781, 501.058349609375, 501.06890869140625, 501.097412109375, 501.1131896972656, 501.1316833496094, 501.1764221191406, 501.2341003417969, 501.25738525390625, 501.26953125, 501.2843322753906, 501.2987060546875, 501.315673828125, 501.4146423339844, 501.4366455078125, 501.4476623535156, 501.4614562988281, 501.50021362304

In [ ]:
raw = RawFile('01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2.raw')
# Get the scan numbers from the msms file and save the scan + info in a dictionary
from fisher_py.data.business import Scan
import numpy as np
scan_mzs = []
scan_ints = []
scan_mass_analyzers = []
scan_collison_energy = []
for scan in msms['Scan number']:
  raw_scan = Scan.from_file(raw._raw_file_access, scan)
  scan_mzs.append(np.array(raw_scan.preferred_masses))
  scan_ints.append(np.array(raw_scan.preferred_intensities))
  scan_mass_analyzers.append(raw_scan.scan_type.split(' + ')[0])
  frag_infos = [f.split(' ')[0] for f in raw_scan.scan_type.split('@')[1:]]
  splits = [[i for i, g in enumerate(f) if g.isnumeric()][0] for f in frag_infos]
  NCEs = [float(frag[split:]) for split, frag in zip(splits, frag_infos)]
  scan_collison_energy.append(NCEs[0])

annotation_df = pd.DataFrame(msms[['Modified sequence', 'Charge', 'Scan number', 'Length']].values, columns=['MODIFIED_SEQUENCE', 'PRECURSOR_CHARGE', 'SCAN_NUMBER', 'PEPTIDE_LENGTH'])
annotation_df['MZ'] = scan_mzs
annotation_df['INTENSITIES'] = scan_ints
annotation_df['MASS_ANALYZER'] = scan_mass_analyzers
annotation_df['COLLISION_ENERGY'] = scan_collison_energy

from spectrum_fundamentals.mod_string import maxquant_to_internal
annotation_df['MODIFIED_SEQUENCE'] = maxquant_to_internal(annotation_df['MODIFIED_SEQUENCE'].values)

from spectrum_fundamentals.annotation.annotation import annotate_spectra
annotation = annotate_spectra(annotation_df)

